In [1]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Installing findspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Installing pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 199 kB 48.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=dec870ae0bfcb77e043eb924a7c500b3c701d2b4aa8adb44b10de38d8e2f67a8
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [4]:
# Installing java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [5]:
# Installing spark
!wget https://downloads.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz

--2022-07-31 10:06:26--  https://downloads.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.95.219, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274079476 (261M) [application/x-gzip]
Saving to: ‘spark-3.3.0-bin-hadoop2.tgz’

spark-3.3.0-bin-had 100%[===================>] 261.38M  28.2MB/s    in 9.9s    

2022-07-31 10:06:37 (26.5 MB/s) - ‘spark-3.3.0-bin-hadoop2.tgz’ saved [274079476/274079476]



In [6]:
import findspark
findspark.init()

In [7]:
import pyspark.sql.functions as pyf

In [8]:
# Creating a spark session
from pyspark.sql import DataFrame, SparkSession
spark = SparkSession.builder.appName("House Price Prediction").getOrCreate()
spark

In [9]:
# Specifying the file path
file_location = "/content/drive/MyDrive/ColabNotebooks/test.csv"
file_type = "csv"
# CSV options
infer_schema = True
first_row_is_header = True
delimiter = ","

In [10]:
# The applied options are for CSV files. For other file types, these will be ignored.
DF = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [11]:
# Reviewing the dataset
DF.show(10, False)

+----+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
|Id  |MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgTy

In [12]:
# Printing the info/schema 
DF.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [13]:
# Checking for null values
from pyspark.sql.functions import isnull, when, count, col
DF.select([count(when(isnull(c), c)).alias(c) for c in DF.columns]).show()

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgType

In [14]:
# Confirming that there are null values in the dataset
DF.distinct().show()

+----+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
|  Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgTy

In [15]:
# Replacing NA to null values for imputation
def fixing_null_values(DF, col_name):
  if col_name != 'MasVnrType' and col_name != 'Electrical':
    new_DF = DF.withColumn(col_name, when((col(col_name)=='NA'), None).otherwise(col(col_name)).cast("float"))
  else:
    new_DF = DF.withColumn(col_name, when((col(col_name)=='NA'), None).otherwise(col(col_name)))
  return new_DF

DF2 = fixing_null_values(DF, 'LotFrontage')
DF3 = fixing_null_values(DF2, 'MasVnrType')
DF4 = fixing_null_values(DF3, 'MasVnrArea')
DF5 = fixing_null_values(DF4, 'Electrical')
DF6 = fixing_null_values(DF5, 'GarageYrBlt')
DF6.show()

+----+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
|  Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgTy

In [16]:
# Confirming that there are null values in the dataset
DF6.distinct().show()

+----+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
|  Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgTy

In [17]:
from pyspark.sql.functions import isnull, when, count, col
DF6.select([count(when(isnull(c), c)).alias(c) for c in DF6.columns]).show()

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgType

In [18]:
# Null value imputation
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols=['LotFrontage', 'MasVnrArea',  'GarageYrBlt'], outputCols=["{}_imputed".format(c) for c in ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']]).setStrategy("median")

In [19]:
imputer.fit(DF6).transform(DF6).show()

+----+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+-------------------+------------------+-------------------+
|  Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|Lo

In [20]:
# Replacing the null values in categorical columns
DF7 = DF6.na.fill("None",["MasVnrType"]).na.fill("SBrkr",["Electrical"])


In [21]:
# Dropping these columns post null value imputation
DF8 = DF7.drop('LotFrontage', 'MasVnrArea',  'GarageYrBlt')

In [22]:
DF8.show()

+----+----------+--------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
|  Id|MSSubClass|MSZoning|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgType|HouseStyle|OverallQual|OverallCond|YearBuilt

In [23]:
# Checking for null values one last time - no null values found
from pyspark.sql.functions import isnull, when, count, col
DF8.select([count(when(isnull(c), c)).alias(c) for c in DF8.columns]).show()

+---+----------+--------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+
| Id|MSSubClass|MSZoning|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgType|HouseStyle|OverallQual|OverallCond|YearBuilt|Y

In [24]:
# Checking for duplicate values - no duplicate values found
DF8.groupBy(DF8.columns).count().filter("count > 1").show()

+---+----------+--------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+-----+
| Id|MSSubClass|MSZoning|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgType|HouseStyle|OverallQual|OverallCond|YearB

In [25]:
DF.columns

['Id',
 'MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF'

In [26]:
DF.summary()

DataFrame[summary: string, Id: string, MSSubClass: string, MSZoning: string, LotFrontage: string, LotArea: string, Street: string, Alley: string, LotShape: string, LandContour: string, Utilities: string, LotConfig: string, LandSlope: string, Neighborhood: string, Condition1: string, Condition2: string, BldgType: string, HouseStyle: string, OverallQual: string, OverallCond: string, YearBuilt: string, YearRemodAdd: string, RoofStyle: string, RoofMatl: string, Exterior1st: string, Exterior2nd: string, MasVnrType: string, MasVnrArea: string, ExterQual: string, ExterCond: string, Foundation: string, BsmtQual: string, BsmtCond: string, BsmtExposure: string, BsmtFinType1: string, BsmtFinSF1: string, BsmtFinType2: string, BsmtFinSF2: string, BsmtUnfSF: string, TotalBsmtSF: string, Heating: string, HeatingQC: string, CentralAir: string, Electrical: string, 1stFlrSF: string, 2ndFlrSF: string, LowQualFinSF: string, GrLivArea: string, BsmtFullBath: string, BsmtHalfBath: string, FullBath: string, H

In [27]:
columnList = [item[0] for item in DF8.dtypes if item[1].startswith('string')]

In [28]:
output_column_list = list(map(lambda x: x+"_index", columnList))

In [29]:
# String indexer
from pyspark.ml.feature import StringIndexer, OneHotEncoder
indexers = StringIndexer(inputCols=columnList, 
                         outputCols=output_column_list)
strindexedDF = indexers.fit(DF8).transform(DF8)
DF9 = strindexedDF.select("*")


In [30]:
DF9.show()

+----+----------+--------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+--------------+------------+-----------+--------------+-----------------+---------------+---------------+---------------+------------------+----------------+----------------+-------------

In [31]:
# Dropping columns post String Indexing
DF10 = DF9.drop(*columnList)

In [32]:
DF10.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- 1stFlrSF: integer (nullable = true)
 |-- 2ndFlrSF: integer (nullable = true)
 |-- LowQualFinSF: integer (nullable = true)
 |-- GrLivArea: integer (nullable = true)
 |-- FullBath: integer (nullable = true)
 |-- HalfBath: integer (nullable = true)
 |-- BedroomAbvGr: integer (nullable = true)
 |-- KitchenAbvGr: integer (nullable = true)
 |-- TotRmsAbvGrd: integer (nullable = true)
 |-- Fireplaces: integer (nullable = true)
 |-- WoodDeckSF: integer (nullable = true)
 |-- OpenPorchSF: integer (nullable = true)
 |-- EnclosedPorch: integer (nullable = true)
 |-- 3SsnPorch: integer (nullable = true)
 |-- ScreenPorch: integer (nullable = true)
 |-- PoolArea: integer (nullable = true)
 |-- MiscVal:

In [33]:
DF10.show()

+----+----------+-------+-----------+-----------+---------+------------+--------+--------+------------+---------+--------+--------+------------+------------+------------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+--------------+------------+-----------+--------------+-----------------+---------------+---------------+---------------+------------------+----------------+----------------+--------------+----------------+---------------+--------------+-----------------+-----------------+----------------+---------------+---------------+----------------+--------------+--------------+------------------+------------------+----------------+------------------+----------------+---------------+-----------------+-------------+---------------+----------------+----------------+------------------+------------------+-----------------+----------------+-----------------+----------------+------------------+----------------+----------------+----------

In [34]:
DF10.columns

['Id',
 'MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'MSZoning_index',
 'Street_index',
 'Alley_index',
 'LotShape_index',
 'LandContour_index',
 'Utilities_index',
 'LotConfig_index',
 'LandSlope_index',
 'Neighborhood_index',
 'Condition1_index',
 'Condition2_index',
 'BldgType_index',
 'HouseStyle_index',
 'RoofStyle_index',
 'RoofMatl_index',
 'Exterior1st_index',
 'Exterior2nd_index',
 'MasVnrType_index',
 'ExterQual_index',
 'ExterCond_index',
 'Foundation_index',
 'BsmtQual_index',
 'BsmtCond_index',
 'BsmtExposure_index',
 'BsmtFinType1_index',
 'BsmtFinSF1_index',
 'BsmtFinType2_index',
 'BsmtFinSF2_index',
 'BsmtUnfSF_index',
 'TotalBsmtSF_index',
 'Heating_inde

In [35]:
input_cols = ['MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [36]:
output_cols = list(map(lambda x: x+"_scaled", input_cols))

In [37]:
# Vectorizing the independent features
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=input_cols,outputCol="Independent_Features")

In [38]:
DF11 = featureassembler.transform(DF10)

In [39]:
DF11.show()

+----+----------+-------+-----------+-----------+---------+------------+--------+--------+------------+---------+--------+--------+------------+------------+------------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+--------------+------------+-----------+--------------+-----------------+---------------+---------------+---------------+------------------+----------------+----------------+--------------+----------------+---------------+--------------+-----------------+-----------------+----------------+---------------+---------------+----------------+--------------+--------------+------------------+------------------+----------------+------------------+----------------+---------------+-----------------+-------------+---------------+----------------+----------------+------------------+------------------+-----------------+----------------+-----------------+----------------+------------------+----------------+----------------+----------

In [40]:
# Import StandardScaler from pyspark.ml.feature package
from pyspark.ml.feature import StandardScaler

# Create the StandardScaler object. It only take feature column (dense vector)
stdscaler = StandardScaler(inputCol="Independent_Features", outputCol="Scaled_Features")

# Fit the StandardScaler object on the output of the dense vector data and transform
stdscaledDF = stdscaler.fit(DF11).transform(DF11)
stdscaledDF.select("*").show()

+----+----------+-------+-----------+-----------+---------+------------+--------+--------+------------+---------+--------+--------+------------+------------+------------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+--------------+------------+-----------+--------------+-----------------+---------------+---------------+---------------+------------------+----------------+----------------+--------------+----------------+---------------+--------------+-----------------+-----------------+----------------+---------------+---------------+----------------+--------------+--------------+------------------+------------------+----------------+------------------+----------------+---------------+-----------------+-------------+---------------+----------------+----------------+------------------+------------------+-----------------+----------------+-----------------+----------------+------------------+----------------+----------------+----------

In [41]:
DF12 = stdscaledDF.drop(*input_cols, "Independent_Features")

In [42]:
DF12.show()

+----+--------------+------------+-----------+--------------+-----------------+---------------+---------------+---------------+------------------+----------------+----------------+--------------+----------------+---------------+--------------+-----------------+-----------------+----------------+---------------+---------------+----------------+--------------+--------------+------------------+------------------+----------------+------------------+----------------+---------------+-----------------+-------------+---------------+----------------+----------------+------------------+------------------+-----------------+----------------+-----------------+----------------+------------------+----------------+----------------+----------------+----------------+----------------+------------+-----------+-----------------+--------------+-------------------+--------------------+
|  Id|MSZoning_index|Street_index|Alley_index|LotShape_index|LandContour_index|Utilities_index|LotConfig_index|LandSlope_ind

In [43]:
DF12.columns

['Id',
 'MSZoning_index',
 'Street_index',
 'Alley_index',
 'LotShape_index',
 'LandContour_index',
 'Utilities_index',
 'LotConfig_index',
 'LandSlope_index',
 'Neighborhood_index',
 'Condition1_index',
 'Condition2_index',
 'BldgType_index',
 'HouseStyle_index',
 'RoofStyle_index',
 'RoofMatl_index',
 'Exterior1st_index',
 'Exterior2nd_index',
 'MasVnrType_index',
 'ExterQual_index',
 'ExterCond_index',
 'Foundation_index',
 'BsmtQual_index',
 'BsmtCond_index',
 'BsmtExposure_index',
 'BsmtFinType1_index',
 'BsmtFinSF1_index',
 'BsmtFinType2_index',
 'BsmtFinSF2_index',
 'BsmtUnfSF_index',
 'TotalBsmtSF_index',
 'Heating_index',
 'HeatingQC_index',
 'CentralAir_index',
 'Electrical_index',
 'BsmtFullBath_index',
 'BsmtHalfBath_index',
 'KitchenQual_index',
 'Functional_index',
 'FireplaceQu_index',
 'GarageType_index',
 'GarageFinish_index',
 'GarageCars_index',
 'GarageArea_index',
 'GarageQual_index',
 'GarageCond_index',
 'PavedDrive_index',
 'PoolQC_index',
 'Fence_index',
 'Misc

In [44]:
# Import VectorAssembler from pyspark.ml.feature package
from pyspark.ml.feature import VectorAssembler
# Create a list of all the variables that you want to create feature vectors
# These features are then further used for training model
features_col = ['MSZoning_index',
 'Street_index',
 'Alley_index',
 'LotShape_index',
 'LandContour_index',
 'Utilities_index',
 'LotConfig_index',
 'LandSlope_index',
 'Neighborhood_index',
 'Condition1_index',
 'Condition2_index',
 'BldgType_index',
 'HouseStyle_index',
 'RoofStyle_index',
 'RoofMatl_index',
 'Exterior1st_index',
 'Exterior2nd_index',
 'MasVnrType_index',
 'ExterQual_index',
 'ExterCond_index',
 'Foundation_index',
 'BsmtQual_index',
 'BsmtCond_index',
 'BsmtExposure_index',
 'BsmtFinType1_index',
 'BsmtFinSF1_index',
 'BsmtFinType2_index',
 'BsmtFinSF2_index',
 'BsmtUnfSF_index',
 'TotalBsmtSF_index',
 'Heating_index',
 'HeatingQC_index',
 'CentralAir_index',
 'Electrical_index',
 'BsmtFullBath_index',
 'BsmtHalfBath_index',
 'KitchenQual_index',
 'Functional_index',
 'FireplaceQu_index',
 'GarageType_index',
 'GarageFinish_index',
 'GarageCars_index',
 'GarageArea_index',
 'GarageQual_index',
 'GarageCond_index',
 'PavedDrive_index',
 'PoolQC_index',
 'Fence_index',
 'MiscFeature_index',
 'SaleType_index',
 'SaleCondition_index',
 'Scaled_Features']
# Create the VectorAssembler object
assembler = VectorAssembler(inputCols=features_col, outputCol="independent_features")
DF13 = assembler.transform(DF12)
final_DF = DF13.select('independent_features')

In [45]:
final_DF.show()

+--------------------+
|independent_features|
+--------------------+
|(76,[0,9,20,24,25...|
|(76,[3,6,13,15,16...|
|(76,[3,8,12,21,25...|
|(76,[3,8,12,17,25...|
|(76,[3,4,8,11,15,...|
|(76,[3,6,8,12,15,...|
|(76,[3,8,15,16,19...|
|(76,[3,8,12,21,24...|
|(76,[8,15,16,21,2...|
|(76,[6,15,16,20,2...|
|(76,[0,3,6,11,15,...|
|(76,[0,8,11,12,15...|
|(76,[0,8,11,12,15...|
|(76,[6,8,11,12,15...|
|(76,[6,8,11,15,16...|
|(76,[3,8,12,17,18...|
|(76,[3,6,8,17,18,...|
|(76,[8,9,13,17,18...|
|(76,[3,8,13,15,16...|
|(76,[4,7,8,13,17,...|
+--------------------+
only showing top 20 rows



We will apply the saved models to our cleaned test dataset

In [46]:
# Linear Regression
from pyspark.ml.regression import LinearRegressionModel
lr_model=LinearRegressionModel.load("/content/drive/MyDrive/ColabNotebooks/linearmodel")

In [47]:
# Coefficients
lr_model.coefficients

DenseVector([270.2467, -43388.1032, 392.7587, -911.039, -2625.3131, -65764.4891, 1242.4279, 9041.409, 714.3812, -3560.0153, -11164.3189, 4607.1104, 1243.1306, 4003.8817, -7773.0248, -523.4145, 242.8984, 1003.1548, 11738.5977, 2561.3484, -2991.9038, 4683.5881, -5495.0564, 2953.3575, 2038.8206, -1016.4128, -1252.9206, -1247.6939, 3124.2432, -359.0977, 5567.7165, -5572.0781, 1319.3323, 1134.1041, 2661.2289, 4863.5737, -178.8684, -2522.6594, 36982.5559, -485.1077, -15367.8055, 3329.9908, -1351.1044, -12650.1528, 4013.1611, 20297.8458, 6596.0765, 6150.0566, -319.3888, 2871.3266, 971.2507, -84.3553, 3190.0386, 7729.8117, 10181.3915, -1075.5595, 13569.0168, 3927.7516, 1080.5741, 2329.5801, -296.8889, -4774.8182, -2639.101, 5376.1262, 1726.8075, 9416.3419, 1974.5591, 3026.1021, -428.5109, 101.7383, 708.6965, 3806.3385, -5058.7232, 987.7361, -513.6281, -1414.1311])

In [48]:
# Intercepts
lr_model.intercept

1700744.539585983

In [49]:
# Prediction
pred_results_lr=lr_model.transform(final_DF)

In [50]:
pred_results_lr.show()

+--------------------+-------------------+
|independent_features|         prediction|
+--------------------+-------------------+
|(76,[0,9,20,24,25...|  845997.7623601628|
|(76,[3,6,13,15,16...|  2442642.711030602|
|(76,[3,8,12,21,25...| 1108027.5683511274|
|(76,[3,8,12,17,25...| 1114239.3175449998|
|(76,[3,4,8,11,15,...|   829045.285388798|
|(76,[3,6,8,12,15,...|  1512072.285707364|
|(76,[3,8,15,16,19...| 1846195.9865873605|
|(76,[3,8,12,21,24...| 1546618.5001455755|
|(76,[8,15,16,21,2...| 1683414.4367580926|
|(76,[6,15,16,20,2...|-31814.204898669384|
|(76,[0,3,6,11,15,...| 2348023.3277581464|
|(76,[0,8,11,12,15...|  2251393.172591308|
|(76,[0,8,11,12,15...|  2413056.737204416|
|(76,[6,8,11,12,15...| 1030360.6034420847|
|(76,[6,8,11,15,16...| 3185669.7465107203|
|(76,[3,8,12,17,18...| 1942784.8269041688|
|(76,[3,6,8,17,18,...| 1548715.9846251279|
|(76,[8,9,13,17,18...| 2033848.4403946232|
|(76,[3,8,13,15,16...| 2144683.5089820796|
|(76,[4,7,8,13,17,...| 1441852.1211650486|
+----------

In [51]:
# Random Forest Regression
from pyspark.ml.regression import RandomForestRegressionModel
rf_model=RandomForestRegressionModel.load("/content/drive/MyDrive/ColabNotebooks/rfmodel")

In [52]:
# Prediction
pred_results_rf=rf_model.transform(final_DF)

In [53]:
pred_results_rf.show()

+--------------------+------------------+
|independent_features|        prediction|
+--------------------+------------------+
|(76,[0,9,20,24,25...|131228.31738490847|
|(76,[3,6,13,15,16...|149901.76225418953|
|(76,[3,8,12,21,25...|214464.02321611723|
|(76,[3,8,12,17,25...|218331.25819579203|
|(76,[3,4,8,11,15,...|197843.39694117874|
|(76,[3,6,8,12,15,...|212043.84633681766|
|(76,[3,8,15,16,19...| 213476.9146491426|
|(76,[3,8,12,21,24...|214815.29569579204|
|(76,[8,15,16,21,2...| 188530.6053617329|
|(76,[6,15,16,20,2...|145424.03514691128|
|(76,[0,3,6,11,15,...|177034.22116883117|
|(76,[0,8,11,12,15...|164653.57694325238|
|(76,[0,8,11,12,15...| 157554.8094829349|
|(76,[6,8,11,12,15...|196692.62074051655|
|(76,[6,8,11,15,16...|173151.87882518902|
|(76,[3,8,12,17,18...|224481.62173834624|
|(76,[3,6,8,17,18,...|227446.66245263192|
|(76,[8,9,13,17,18...|215281.58573593077|
|(76,[3,8,13,15,16...|222563.53018037524|
|(76,[4,7,8,13,17,...|211843.68788913987|
+--------------------+------------

In [54]:
# Gradient Boost Regression
from pyspark.ml.regression import GBTRegressionModel
gbr_model=GBTRegressionModel.load("/content/drive/MyDrive/ColabNotebooks/gbrmodel")

In [55]:
pred_results_gb = gbr_model.transform(final_DF)

In [56]:
pred_results_gb.show()

+--------------------+------------------+
|independent_features|        prediction|
+--------------------+------------------+
|(76,[0,9,20,24,25...|135315.87058067514|
|(76,[3,6,13,15,16...| 132325.0483602264|
|(76,[3,8,12,21,25...|182141.64372139517|
|(76,[3,8,12,17,25...|182141.64372139517|
|(76,[3,4,8,11,15,...| 141350.9394908356|
|(76,[3,6,8,12,15,...| 181789.3236056594|
|(76,[3,8,15,16,19...|165201.22290795224|
|(76,[3,8,12,21,24...| 180764.0952022458|
|(76,[8,15,16,21,2...|210636.20740178882|
|(76,[6,15,16,20,2...| 153215.1478162775|
|(76,[0,3,6,11,15,...|161237.89781477398|
|(76,[0,8,11,12,15...|105198.01403120114|
|(76,[0,8,11,12,15...|  70916.7464397179|
|(76,[6,8,11,12,15...|115171.37607784123|
|(76,[6,8,11,15,16...|100528.23045440922|
|(76,[3,8,12,17,18...|168201.58186807795|
|(76,[3,6,8,17,18,...| 167461.9299740029|
|(76,[8,9,13,17,18...|176014.09249675242|
|(76,[3,8,13,15,16...|161569.40085548043|
|(76,[4,7,8,13,17,...|226504.98562544255|
+--------------------+------------

In [57]:
# Extracting output csv file for submission
id_column = DF12.select("id")

In [58]:
pred_column = pred_results_rf.select("prediction")

In [59]:
id_pandas = id_column.toPandas()

In [60]:
pred_pandas = pred_column.toPandas()

In [61]:
type(id_pandas)

pandas.core.frame.DataFrame

In [62]:
import pandas as pd
submission = pd.concat([id_pandas,pred_pandas], axis=1)

In [63]:
submission

,id,prediction
0,1461,131228.317385
1,1462,149901.762254
2,1463,214464.023216
3,1464,218331.258196
4,1465,197843.396941
...,...,...
1454,2915,191480.195253
1455,2916,202974.328847
1456,2917,205377.202997
1457,2918,183841.426753


In [64]:
submission.to_csv("final_submission.csv", index=False)